In [1]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import concurrent.futures

In [2]:
folder="Gyro"
path="C:\\Users\\nilu1\\Desktop\\Code\\AndroidPattern\\Data\\Train\\"+folder+"\\"
pathex=[]
temp = [f for f in listdir(path) if isfile(join(path, f))]
# temp.sort()
# files=["all"]
# files=["66b325","b56522","a99827"]
# files=["66b325","b56522"]
# files=["a29c08","66b325","b56522"]
files=["a29c12","a29c08","66b325","b56522"]
# files=["e0f709","a29c08","66b325","b56522"]
# files=["e0f709"]
# files=["a99828","a99830"]
# files=["a99828","a99830","66b325","b56522"]
for i in range(len(temp)):
    st=temp[i]
    if(os.path.getsize(path+st)<4096):
        continue
    if(temp[i][:6] in files):
        pathex.append(temp[i])
#     if(files[0]=="all"):
#         pathex.append(temp[i])
# print(pathex)
print(len(temp))
print(len(pathex))

7985
2325


In [3]:
# folder_n_no.ofvalues_no.offiles_keycolno.
propath="C:\\Users\\nilu1\\Desktop\\Code\\AndroidPattern\\Data\\Processed\\"
def savefile():
    df.to_csv (propath+folder+"_"+str(n)+"_"+fcode+"_"+"&".join(files)+"_"+str(keypos)+".csv", index = False, header=True)

In [4]:
#taking into account that error might occur while reading a file (putting try and except)
# n -> number of lines in csv file
# actual -> the actual key pressed
# minkeyd -> index of nearest key down time
# minkeyu -> index of nearest key up time
# keydt -> key down time from file name
# keyut -> key up time from file name
fcode="wininituni"
n=400
keypos=1
actual=1
df=pd.read_csv(path+pathex[0])
for i in range(len(pathex)):
    try:
        #removing initial values
#         print(pathex[i])
        dff=pd.read_csv(path+pathex[i])
        for j in dff.index:
            if(dff["Time"][j]>10):
                break
        winerr=dff["Time"][j]
        dff=dff.tail(dff.size-j)
        dff.reset_index(inplace=True)
        dff=dff.drop(columns="index")
        
#         dfuni = pd.DataFrame(columns = ["Time","X","Y","Z"])
# dff["Time"].iloc[-1]
        dfuni = pd.DataFrame(0.0, index=np.arange(dff["Time"].iloc[-1]+2), columns=["X","Y","Z"])
        k=0
        sumx=0
        sumy=0
        sumz=0
        for j in dfuni.index:
            while(k<dff.index[-1] and dff["Time"][k]==dff["Time"][k+1]):
                k+=1
            if(k>dff.index[-1]):
                break
#             print("j : ",j)
#             print("dff Time : ",dff["Time"][k])
#             print("dff : ",dff["Time"][k]-dff["Time"][0])
            if(j==dff["Time"][k]-dff["Time"][0]):
#                 print("match")
                dfuni["X"][j]=dff["X"][k]
                dfuni["Y"][j]=dff["Y"][k]
                dfuni["Z"][j]=dff["Z"][k]
                k+=1
            else:
#                 print("no match")
                dfuni["X"][j]=dfuni["X"][j-1]
                dfuni["Y"][j]=dfuni["Y"][j-1]
                dfuni["Z"][j]=dfuni["Z"][j-1]
            if(j<300):
                sumx+=dff["X"][j]
                sumy+=dff["Y"][j]
                sumz+=dff["Z"][j]
#         print(dff["Time"][k-1])
        dfuni=dfuni.head(j)
#         print(dfuni)
        
        keydt=pathex[i].rfind("_")+4
        keyut=pathex[i].rfind("&")+1
        minkeyd=int(pathex[i][keydt:keyut-1])+winerr
        minkeyu=int(pathex[i][keyut:-4])+winerr
        
        dff=dfuni
        
#         sumx=0
#         sumy=0
#         sumz=0
#         for j in range(300):
#             sumx+=dff["X"][j]
#             sumy+=dff["Y"][j]
#             sumz+=dff["Z"][j]
        avg=[sumx/300,sumy/300,sumz/300]
        
        for j in dff.index:
            dff['X'][j]=abs(dff['X'][j]-avg[0])
            dff['Y'][j]=abs(dff['Y'][j]-avg[1])
            dff['Z'][j]=abs(dff['Z'][j]-avg[2])
#         print(dff)
        
        window=minkeyu-minkeyd
        left=int(n/2)-window
        dff=dff.head(minkeyu + left)
        dff=dff.tail(n)
        dff.reset_index(inplace=True)
        dff=dff.drop(columns="index")
        df1=dff.stack().swaplevel()        
        df1.index=df1.index.map('{0[0]}_{0[1]}'.format) 
        dff=df1.to_frame().T
        ind=pathex[i].rfind("_")+3
        print(i,pathex[i])
        dff.insert(n*3, "key", int(pathex[i][ind]))
        keypos=n*3
        if(i==0):
            df=dff
        else:
            df=df.append(dff,ignore_index = True)
    except:
        print("error : ",pathex[i])
#         traceback.print_exec()
        continue

0 66b325-03-2020_02_10_19_PM1992&1078.csv
1 66b325-03-2020_02_10_21_PM0844&944.csv
2 66b325-03-2020_02_10_23_PM1753&840.csv
3 66b325-03-2020_02_10_25_PM4866&959.csv
4 66b325-03-2020_02_10_28_PM6818&885.csv
5 66b325-03-2020_02_10_30_PM9825&879.csv
6 66b325-03-2020_02_10_32_PM4826&932.csv
7 66b325-03-2020_02_10_34_PM6900&967.csv
8 66b325-03-2020_02_10_36_PM1740&836.csv
9 66b325-03-2020_02_10_38_PM9729&831.csv
10 66b325-03-2020_02_10_41_PM3788&874.csv
11 66b325-03-2020_02_10_43_PM0804&890.csv
12 66b325-03-2020_02_10_45_PM7797&887.csv
13 66b325-03-2020_02_10_47_PM4729&822.csv
14 66b325-03-2020_02_10_49_PM9802&873.csv
15 66b325-03-2020_02_10_51_PM9741&819.csv
16 66b325-03-2020_02_10_53_PM0763&848.csv
17 66b325-03-2020_02_10_56_PM1804&868.csv
18 66b325-03-2020_02_10_58_PM0721&807.csv
19 66b325-03-2020_02_11_00_PM4792&881.csv
20 66b325-03-2020_02_11_02_PM3633&705.csv
21 66b325-03-2020_02_11_04_PM5738&818.csv
22 66b325-03-2020_02_11_06_PM5723&814.csv
23 66b325-03-2020_02_11_09_PM81135&1222.csv

192 66b325-03-2020_02_32_39_PM1726&833.csv
193 66b325-03-2020_02_32_41_PM6862&919.csv
194 66b325-03-2020_02_32_43_PM2712&800.csv
195 66b325-03-2020_02_32_45_PM1750&853.csv
196 66b325-03-2020_02_32_48_PM61062&1122.csv
197 66b325-03-2020_02_32_50_PM3930&1005.csv
198 66b325-03-2020_02_32_53_PM31221&1283.csv
199 66b325-03-2020_02_32_55_PM6807&887.csv
200 66b325-03-2020_02_32_57_PM6973&1045.csv
201 66b325-03-2020_02_34_05_PM31008&1039.csv
202 66b325-03-2020_02_34_07_PM3726&799.csv
203 66b325-03-2020_02_34_09_PM6765&874.csv
204 66b325-03-2020_02_34_11_PM7997&1083.csv
205 66b325-03-2020_02_34_14_PM11021&1098.csv
206 66b325-03-2020_02_34_16_PM2914&979.csv
207 66b325-03-2020_02_34_18_PM4834&918.csv
208 66b325-03-2020_02_34_20_PM2650&750.csv
209 66b325-03-2020_02_34_23_PM9788&873.csv
210 66b325-03-2020_02_34_25_PM8831&915.csv
211 66b325-03-2020_02_34_27_PM9740&825.csv
212 66b325-03-2020_02_34_29_PM8827&940.csv
213 66b325-03-2020_02_34_31_PM6803&876.csv
214 66b325-03-2020_02_34_33_PM9744&838.csv


381 66b325-03-2020_02_43_28_PM4749&848.csv
382 66b325-03-2020_02_43_30_PM7767&864.csv
383 66b325-03-2020_02_43_33_PM1789&902.csv
384 66b325-03-2020_02_43_35_PM4764&814.csv
385 66b325-03-2020_02_43_37_PM51098&1214.csv
386 66b325-03-2020_02_43_40_PM41392&1488.csv
387 66b325-03-2020_02_43_42_PM81035&1118.csv
388 66b325-03-2020_02_43_45_PM81257&1354.csv
389 66b325-03-2020_02_43_47_PM2735&802.csv
390 66b325-03-2020_02_43_49_PM3751&816.csv
391 66b325-03-2020_02_43_51_PM3791&833.csv
392 66b325-03-2020_02_43_54_PM9990&1079.csv
393 66b325-03-2020_02_43_56_PM41037&1112.csv
394 66b325-03-2020_02_43_59_PM31179&1237.csv
395 66b325-03-2020_02_44_01_PM8933&1020.csv
396 66b325-03-2020_02_44_04_PM91012&1074.csv
397 66b325-03-2020_02_44_08_PM92753&2847.csv
398 66b325-03-2020_02_44_10_PM9688&773.csv
399 66b325-03-2020_02_44_12_PM5806&917.csv
400 66b325-03-2020_02_44_14_PM5691&797.csv
401 a29c08-04-2020_07_09_43_PM61048&1129.csv
402 a29c08-04-2020_07_09_45_PM6978&1059.csv
403 a29c08-04-2020_07_09_48_PM411

570 a29c08-04-2020_07_17_40_PM1900&933.csv
571 a29c08-04-2020_07_17_42_PM7886&962.csv
572 a29c08-04-2020_07_17_44_PM21100&1144.csv
573 a29c08-04-2020_07_17_46_PM0893&997.csv
574 a29c08-04-2020_07_17_48_PM5802&828.csv
575 a29c08-04-2020_07_17_50_PM5915&1009.csv
576 a29c08-04-2020_07_17_52_PM1831&906.csv
577 a29c08-04-2020_07_17_55_PM01018&1120.csv
578 a29c08-04-2020_07_17_57_PM2894&971.csv
579 a29c08-04-2020_07_17_59_PM7922&979.csv
580 a29c08-04-2020_07_18_01_PM61591&1639.csv
581 a29c08-04-2020_07_18_03_PM2862&907.csv
582 a29c08-04-2020_07_18_06_PM6967&1009.csv
583 a29c08-04-2020_07_18_07_PM2755&819.csv
584 a29c08-04-2020_07_18_09_PM6864&915.csv
585 a29c08-04-2020_07_18_12_PM41015&1116.csv
586 a29c08-04-2020_07_18_14_PM7903&1006.csv
587 a29c08-04-2020_07_18_16_PM4996&1062.csv
588 a29c08-04-2020_07_18_18_PM91049&1123.csv
589 a29c08-04-2020_07_18_20_PM3768&796.csv
590 a29c08-04-2020_07_18_22_PM3891&934.csv
591 a29c08-04-2020_07_18_24_PM4899&962.csv
592 a29c08-04-2020_07_18_26_PM3796&849.c

760 a29c08-04-2020_07_25_18_PM8910&992.csv
761 a29c08-04-2020_07_25_24_PM91186&1245.csv
762 a29c08-04-2020_07_25_26_PM8866&936.csv
763 a29c08-04-2020_07_25_28_PM0919&946.csv
764 a29c08-04-2020_07_25_30_PM9871&926.csv
765 a29c08-04-2020_07_25_32_PM1784&848.csv
766 a29c08-04-2020_07_25_34_PM5714&756.csv
767 a29c08-04-2020_07_25_36_PM8878&916.csv
768 a29c08-04-2020_07_25_38_PM5746&804.csv
769 a29c08-04-2020_07_25_40_PM3793&852.csv
770 a29c08-04-2020_07_25_42_PM1837&917.csv
771 a29c08-04-2020_07_25_44_PM51191&1266.csv
772 a29c08-04-2020_07_25_46_PM5777&800.csv
773 a29c08-04-2020_07_25_48_PM6863&895.csv
774 a29c08-04-2020_07_25_50_PM4895&979.csv
775 a29c08-04-2020_07_25_52_PM3863&884.csv
776 a29c08-04-2020_07_25_54_PM0828&897.csv
777 a29c08-04-2020_07_25_56_PM4827&900.csv
778 a29c08-04-2020_07_25_58_PM7821&881.csv
779 a29c08-04-2020_07_26_00_PM3690&748.csv
780 a29c08-04-2020_07_26_02_PM1759&842.csv
781 a29c08-04-2020_07_26_03_PM4685&743.csv
782 a29c08-04-2020_07_26_05_PM8839&860.csv
783 a29

...

1507 a29c12-04-2020_01_47_56_PM2668&707.csv
1508 a29c12-04-2020_01_47_58_PM3627&649.csv
1509 a29c12-04-2020_01_48_00_PM3642&701.csv
1510 a29c12-04-2020_01_48_02_PM6762&828.csv
1511 a29c12-04-2020_01_48_03_PM6705&781.csv
1512 a29c12-04-2020_01_48_05_PM7825&908.csv
1513 a29c12-04-2020_01_48_07_PM7630&719.csv
1514 a29c12-04-2020_01_48_09_PM7995&1087.csv
1515 a29c12-04-2020_01_48_11_PM8670&744.csv
1516 a29c12-04-2020_01_48_13_PM9979&1028.csv
1517 a29c12-04-2020_01_48_43_PM4904&979.csv
1518 a29c12-04-2020_01_48_45_PM8938&1000.csv
1519 a29c12-04-2020_01_48_46_PM5648&706.csv
1520 a29c12-04-2020_01_48_48_PM2644&670.csv
1521 a29c12-04-2020_01_48_50_PM4845&921.csv
1522 a29c12-04-2020_01_48_52_PM0729&802.csv
1523 a29c12-04-2020_01_48_55_PM3903&915.csv
1524 a29c12-04-2020_01_48_57_PM7805&895.csv
1525 a29c12-04-2020_01_48_58_PM8724&790.csv
1526 a29c12-04-2020_01_49_00_PM2688&753.csv
1527 a29c12-04-2020_01_49_02_PM9846&885.csv
1528 a29c12-04-2020_01_49_04_PM7884&1050.csv
1529 a29c12-04-2020_01_49_06

1693 a29c12-04-2020_01_56_21_PM91010&1095.csv
1694 a29c12-04-2020_01_56_23_PM3704&804.csv
1695 a29c12-04-2020_01_56_25_PM4810&884.csv
1696 a29c12-04-2020_01_56_27_PM21068&1141.csv
1697 a29c12-04-2020_01_56_29_PM9768&862.csv
1698 a29c12-04-2020_01_56_32_PM4932&943.csv
1699 a29c12-04-2020_01_56_34_PM8902&1007.csv
1700 a29c12-04-2020_01_56_36_PM0728&820.csv
1701 a29c12-04-2020_01_56_38_PM7757&848.csv
1702 a29c12-04-2020_01_56_39_PM3653&736.csv
1703 a29c12-04-2020_01_56_41_PM5713&781.csv
1704 a29c12-04-2020_01_56_43_PM4720&776.csv
1705 a29c12-04-2020_01_56_45_PM0764&819.csv
1706 a29c12-04-2020_01_56_47_PM5608&682.csv
1707 a29c12-04-2020_01_56_49_PM6932&984.csv
1708 a29c12-04-2020_01_56_51_PM5704&768.csv
1709 a29c12-04-2020_01_56_53_PM7713&805.csv
1710 a29c12-04-2020_01_56_54_PM5653&736.csv
1711 a29c12-04-2020_01_56_56_PM6743&785.csv
1712 a29c12-04-2020_01_56_58_PM6638&690.csv
1713 a29c12-04-2020_01_57_00_PM8688&752.csv
1714 a29c12-04-2020_01_57_02_PM8667&741.csv
1715 a29c12-04-2020_01_57_0

1879 a29c12-04-2020_02_03_40_PM9714&756.csv
1880 a29c12-04-2020_02_03_42_PM8656&704.csv
1881 a29c12-04-2020_02_03_44_PM4883&966.csv
1882 a29c12-04-2020_02_03_46_PM1749&807.csv
1883 a29c12-04-2020_02_03_48_PM6712&745.csv
1884 a29c12-04-2020_02_03_50_PM7861&938.csv
1885 a29c12-04-2020_02_03_51_PM9633&706.csv
1886 a29c12-04-2020_02_03_53_PM8849&907.csv
1887 a29c12-04-2020_02_03_55_PM1705&778.csv
1888 a29c12-04-2020_02_03_57_PM41014&1084.csv
1889 a29c12-04-2020_02_04_00_PM5918&991.csv
1890 a29c12-04-2020_02_04_01_PM4792&849.csv
1891 a29c12-04-2020_02_04_03_PM6716&793.csv
1892 a29c12-04-2020_02_04_05_PM1816&891.csv
1893 a29c12-04-2020_02_04_07_PM5735&812.csv
1894 a29c12-04-2020_02_04_09_PM91107&1206.csv
1895 a29c12-04-2020_02_04_11_PM3740&804.csv
1896 a29c12-04-2020_02_04_13_PM1920&986.csv
1897 a29c12-04-2020_02_04_15_PM6713&773.csv
1898 a29c12-04-2020_02_04_17_PM6794&868.csv
1899 a29c12-04-2020_02_04_19_PM5842&890.csv
1900 a29c12-04-2020_02_04_21_PM81137&1210.csv
1901 a29c12-04-2020_02_04_

2065 a29c12-04-2020_12_54_15_PM0838&943.csv
2066 a29c12-04-2020_12_54_17_PM2613&681.csv
2067 a29c12-04-2020_12_54_19_PM1694&769.csv
2068 a29c12-04-2020_12_54_21_PM4912&967.csv
2069 a29c12-04-2020_12_54_23_PM4808&873.csv
2070 a29c12-04-2020_12_54_25_PM5640&704.csv
2071 a29c12-04-2020_12_54_27_PM7933&986.csv
2072 a29c12-04-2020_12_54_29_PM9821&882.csv
2073 a29c12-04-2020_12_54_31_PM6803&869.csv
2074 a29c12-04-2020_12_54_33_PM61242&1265.csv
2075 a29c12-04-2020_12_54_35_PM9761&810.csv
2076 a29c12-04-2020_12_54_37_PM3712&804.csv
2077 a29c12-04-2020_12_54_39_PM0886&973.csv
2078 a29c12-04-2020_12_54_41_PM0878&955.csv
2079 a29c12-04-2020_12_54_59_PM5740&807.csv
2080 a29c12-04-2020_12_55_01_PM9711&801.csv
2081 a29c12-04-2020_12_55_02_PM5658&727.csv
2082 a29c12-04-2020_12_55_04_PM2633&725.csv
2083 a29c12-04-2020_12_55_06_PM9653&707.csv
2084 a29c12-04-2020_12_55_08_PM1710&781.csv
2085 a29c12-04-2020_12_55_10_PM1935&1009.csv
2086 a29c12-04-2020_12_55_12_PM3701&769.csv
2087 a29c12-04-2020_12_55_14_

2254 b56522-03-2020_094536_AM2628&717.csv
2255 b56522-03-2020_094538_AM8738&809.csv
2256 b56522-03-2020_094540_AM5601&697.csv
2257 b56522-03-2020_094542_AM5713&814.csv
2258 b56522-03-2020_094544_AM8647&751.csv
2259 b56522-03-2020_094546_AM7816&924.csv
2260 b56522-03-2020_094549_AM7813&899.csv
2261 b56522-03-2020_094551_AM3666&771.csv
2262 b56522-03-2020_094553_AM3854&910.csv
2263 b56522-03-2020_094555_AM3619&707.csv
2264 b56522-03-2020_094557_AM7759&838.csv
2265 b56522-03-2020_094559_AM3713&803.csv
2266 b56522-03-2020_094601_AM4698&758.csv
2267 b56522-03-2020_094603_AM2647&725.csv
2268 b56522-03-2020_094605_AM7781&844.csv
2269 b56522-03-2020_094607_AM9613&719.csv
2270 b56522-03-2020_094609_AM8603&700.csv
2271 b56522-03-2020_094611_AM1679&741.csv
2272 b56522-03-2020_094613_AM1733&783.csv
2273 b56522-03-2020_094615_AM9708&806.csv
2274 b56522-03-2020_094618_AM4745&817.csv
2275 b56522-03-2020_094620_AM9732&811.csv
2276 b56522-03-2020_094622_AM4596&737.csv
2277 b56522-03-2020_094624_AM2697&

In [5]:
savefile()